In [1]:
from config import OPENAI_API_KEY, TAVILY_API_KEY, LANGCHAIN_TRACING_V2, LANGCHAIN_API_KEY

In [2]:
from typing import Annotated, List, Tuple, Union

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
python_repl_tool = PythonREPLTool()

In [3]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [4]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

Create Agent Supervisor
It will use function calling to choose the next worker node OR finish processing.

In [5]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4-1106-preview")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [6]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


research_agent = create_agent(llm, [tavily_tool], "You are a web researcher.")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
code_agent = create_agent(
    llm,
    [python_repl_tool],
    "You may generate safe python code to analyze data and generate charts using matplotlib.",
)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)


In [7]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [8]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Code hello world and print it to the terminal")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Coder'}}
----


Python REPL can execute arbitrary code. Use with caution.


{'Coder': {'messages': [HumanMessage(content='The code has been executed, and "Hello, World!" has been printed to the terminal.', name='Coder')]}}
----
{'supervisor': {'next': 'FINISH'}}
----


In [9]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Write a brief research report on pikas.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='Pikas are small, round, ovate-bodied mammals adapted to live in the harsh environments of mountainous areas. They belong to the genus Ochotona, the sole living genus of the family Ochotonidae. Pikas are known for their ability to thrive in cold, rocky, and treacherous habitats where other mammals rarely venture. They are particularly sensitive to high temperatures and can perish after only a few hours of exposure to temperatures around 78 degrees Fahrenheit (25.5 degrees Celsius).\n\nPikas do not hibernate; instead, they remain active throughout the winter, traveling in tunnels under rocks and snow, and consuming dried plants they have stored in a process known as "haying." They feed year-round, but their haying activity is limited to summer months. Pikas have two foraging methods: direct consumption and gathering of plants to store for winter. They prefer foraging in temperatures below 25 de

In [10]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Write a brief research report on italian market for shipping.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='## Italian Market for Shipping - 2023 Research Report\n\n### Overview\nThe Italian shipping and logistics market is experiencing various challenges and transformations in 2023. With a backdrop of economic fluctuations and market pressures, the sector is adjusting to new conditions.\n\n### Economic Context\n- **GDP Growth:** The Italian economy is expected to grow by 0.7% in 2023, despite a -0.4% decrease in GDP in Q2 2023 compared to the previous quarter. Year over year, there was a 0.4% increase. The high inflation and interest rates continue to impact the economy, but some signals of improvement are on the horizon.\n\n### E-commerce\n- **Market Decline:** The Italian e-commerce market saw a decline of 8.6% year-over-year in 2022, with all subsegments experiencing a downturn.\n\n### Exports\n- **Food and Beverage:** Italian food and beverage exports, particularly wine, have surged in Q1 2023

In [12]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Tell me who is Davide Malaguti - the italian coacher born in Bologna")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='Davide Malaguti is an Italian entrepreneur and business coach from Bologna. He has been teaching for over 20 years on how to maximize motivation and productivity for individuals at any level within a company. He is described as a visionary, a father, a trainer, an entrepreneur, and a life-coach. Throughout his career, more than 100,000 people have experienced the warmth, humor, and transformative power of his corporate and personal development events.\n\nHe has been a coach and trainer for over 25 years, currently leading seven companies, has published three books, and invented the SPEED methodology, which he has used to train and coach over 100,000 individuals. He is also recognized as an author and the creator of personal growth courses.\n\nFor more detailed information, you can visit his [official website](https://www.davidemalaguti.com/), check out his [LinkedIn profile](https://it.linked